In [ ]:
!pip install Flask
!pip install pyngrok
!pip install ngrok-flask-cart
!pip install peft
!pip install opencc

In [ ]:
from pyngrok import ngrok
from flask import Flask, jsonify, request
from ngrok_flask_cart import run_with_ngrok

ngrok_key = "2lgYcgeCIYuw5aEFkPDhzXDDy6v_26UBnBYfXbrBkxmtS2smp"
port = 5000
ngrok.update()
ngrok.set_auth_token(ngrok_key)

In [ ]:
# Summarization Model
from peft import PeftConfig, PeftModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

summarization_config = PeftConfig.from_pretrained("JamesLin0219/t5LoRASummarization")
summarization_model = PeftModel.from_pretrained(model, "JamesLin0219/t5LoRASummarization")
summarization_tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# New Classification Model
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "yenrong/classification_model_ver2"
classification_model = AutoModelForSequenceClassification.from_pretrained(model_name)
classification_tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# QA model
from transformers import pipeline
import torch

model_id = "vanessasml/cyber-risk-llama-3-8b"

pipe = pipeline(
    "text-generation",
    model = model_id,
    model_kwargs = {"torch_dtype": torch.float16},
    device = "cuda",
)

terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
import torch

app = Flask(__name__)
run_with_ngrok(app=app, domain='--domain=ewe-happy-centrally.ngrok-free.app', auth_token=ngrok_key)

@app.route("/hello", methods=['GET'])
def hello():
    return "Hello World!"

@app.route("/json", methods=['GET'])
def json():
    response_object = {'status': "success", 'message': "example"}
    return jsonify(response_object)

@app.route("/summarize", methods=['POST'])
def summarize():
    post_data = request.get_json()
    response_object = {'status': "success"}

    try:
        paragraph = "summarize: " + post_data['paragraph']
        inputs = summarization_tokenizer(paragraph, return_tensors = "pt").input_ids
        outputs = summarization_model.generate(input_ids = inputs, max_new_tokens = 512, do_sample = True)
        response_object['summary'] = summarization_tokenizer.batch_decode(outputs, skip_special_tokens = True)[0]

    except Exception as e:
        response_object['status'] = "failure"
        response_object['summary'] = str(e)
        response_object['translated_summary'] = str(e)

    return jsonify(response_object)

@app.route("/classify", methods=['POST'])
def classify():
    post_data = request.get_json()
    response_object = {'status': "success"}

    id2label = {
        0: 'cowrie.session.connect',
        1: 'cowrie.client.version',
        2: 'cowrie.client.kex',
        3: 'cowrie.login.failed',
        4: 'cowrie.session.closed',
        5: 'cowrie.login.success',
        6: 'cowrie.session.params',
        7: 'cowrie.command.input',
        8: 'cowrie.session.file_download',
        9: 'cowrie.direct-tcpip.request',
        10: 'cowrie.direct-tcpip.data',
        11: 'cowrie.log.closed',
        12: 'cowrie.command.failed',
        13: 'cowrie.client.size',
        14: 'cowrie.session.file_upload',
        15: 'cowrie.session.file_download.failed'
    }

    col_names = ["username", "input", "size", "compCS", "width",
            "outfile", "protocol", "duration", "height", "url",
            "keyAlgs", "ttylog", "data", "sensor", "arch",
            "session", "shasum", "message", "langCS", "timestamp",
            "kexAlgs", "encCS", "password", "version", "dst_port",
            "macCS", "destfile", "client_fingerprint", "filename"]

    input_text = ""

    try:
        input_text += " ".join(str(post_data.get(column, "none")) for column in col_names)
        prefix = "classify: "
        inputs = classification_tokenizer(prefix + input_text, truncation = True, max_length = 512, return_tensors = "pt")

        with torch.no_grad():
            logits = classification_model(**inputs).logits
        predicted_class_id = logits.argmax().item()

        attack_type = id2label[predicted_class_id]
        response_object['attack_type'] = attack_type

        prob_list = torch.softmax(logits, dim = 1)[0].tolist()
        response_object['probabilities'] = prob_list

    except Exception as e:
        response_object['status'] = "failure"
        response_object['attack_type'] = str(e)

    return jsonify(response_object)

@app.route("/qa", methods=['POST'])
def qa():
    post_data = request.get_json()
    response_object = {'status': "success"}

    try:
        question = post_data['question']
        messages = [
            {"role": "system", "content": "You are an expert of cybersecurity and information security."},
            {"role": "user", "content": question + " Please reply within 260 words."},
        ]

        prompt = pipe.tokenizer.apply_chat_template(
            messages,
            tokenize = False,
            add_generation_prompt = True
        )

        outputs = pipe(
            prompt,
            max_new_tokens = 360,
            eos_token_id = terminators,
            do_sample = True,
            temperature = 0.1,
            top_p = 0.9,
        )
        response_object['answer'] = outputs[0]["generated_text"][len(prompt):]

    except Exception as e:
        response_object['status'] = "failure"
        response_object['answer'] = str(e)

    return jsonify(response_object)

app.run(port = port)